<a href="https://colab.research.google.com/github/lilnoes/notebooks/blob/main/nn_text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
# !pip install --upgrade tensorflow
# !pip install tensorflow-gpu
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Embedding, LSTM, Bidirectional, GRU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import numpy as np
import os
print(tf.__version__)

2.1.0


In [ ]:
link = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

In [ ]:
path = keras.utils.get_file('shakespeare.txt', link)

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
dataset = tf.data.TextLineDataset(path)

In [ ]:
a = set()
for i in dataset.cache():
  a.update(list(i.numpy().decode()))
# a = set()
# dataset.take(3).reduce('0', lambda s,x:s+x)
uniqs = a

In [ ]:
vocab = a
len(vocab)

64

In [ ]:
char2idx = {i:u for u,i in enumerate(vocab)}
idx2char = np.array(list(vocab))

In [ ]:
char2idx['F'],idx2char[24]

(24, 'F')

In [ ]:
encoded_ds = tf.data.Dataset.from_generator(gen, tf.int64, ())
train_ds = encoded_ds.batch(100,True).map(lambda x: (x[:-1], x[1:]))

In [ ]:
train_ds

<MapDataset shapes: ((99,), (99,)), types: (tf.int64, tf.int64)>

In [ ]:
train_data = train_ds.shuffle(10000).batch(64, True)
train_data = train_data.cache()

In [ ]:
train_data = train_data.cache()
for i in train_data:
  pass

In [ ]:
for i in train_data.cache():
  pass

In [ ]:
# for x,y in train_data.cache().take(1):
#   print(model(x))
train_data

<BatchDataset shapes: (<unknown>, <unknown>), types: (tf.int64, tf.int64)>

In [ ]:
vocab_size = len(vocab)
e_dim = 256
rnn_units = 1024
batch_size = 64
model = get_model(vocab_size, e_dim, rnn_units, batch_size)
model.compile(optimizer='adam', loss=loss)
model.summary()

Model: "model_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_40 (InputLayer)        [(64, None)]              0         
_________________________________________________________________
embedding_43 (Embedding)     (64, None, 256)           16384     
_________________________________________________________________
gru_33 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_64 (Dense)             (64, None, 64)            65600     
Total params: 4,020,288
Trainable params: 4,020,288
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_data, epochs=10, callbacks=[ckpt_callback])

Epoch 1/10
168/168 [==============================] - 7s 45ms/step - loss: 1.2300
Epoch 2/10
168/168 [==============================] - 7s 43ms/step - loss: 1.1831
Epoch 3/10
168/168 [==============================] - 7s 43ms/step - loss: 1.1338
Epoch 4/10
168/168 [==============================] - 7s 43ms/step - loss: 1.0846
Epoch 5/10
168/168 [==============================] - 7s 43ms/step - loss: 1.0432
Epoch 6/10
168/168 [==============================] - 7s 43ms/step - loss: 1.0233
Epoch 7/10
168/168 [==============================] - 7s 43ms/step - loss: 1.0022
Epoch 8/10
168/168 [==============================] - 7s 43ms/step - loss: 0.9744
Epoch 9/10
168/168 [==============================] - 7s 43ms/step - loss: 0.9497
Epoch 10/10
168/168 [==============================] - 7s 43ms/step - loss: 0.9348


In [ ]:
def gen():
  for i in dataset:
    for c in list(i.numpy().decode()):
      yield char2idx[c]

In [ ]:
def split_input(x):
  return x[:-1], x[1:]
def decode_sent(x):
  return ''.join(idx2char [list(x.numpy())])

In [ ]:
def loss(labels, preds):
  return keras.losses.sparse_categorical_crossentropy(labels, preds, True)

In [ ]:
ckpt = './training_checkpoints'
prefix = os.path.join(ckpt, 'ckpt_{epoch}')
ckpt_callback = keras.callbacks.ModelCheckpoint(prefix, save_weights_only=True)

In [ ]:
def get_model(vocab_size, e_dim, rnn_units, batch_size):

  inputs = Input(batch_shape=(batch_size,None))
  embedding = Embedding(vocab_size, e_dim)(inputs)
  rnn = GRU(rnn_units, stateful=True, return_sequences=True)(embedding)
  # rnn1 = GRU(rnn_units, stateful=True, return_sequences=True)(rnn)
  # dense1 = Dense(512, activation='relu')(rnn1)
  # dense2 = Dense(256, activation='relu')(dense1)
  preds = Dense(vocab_size)(rnn)
  model = keras.models.Model(inputs, preds)
  return model

In [ ]:
model_pred = get_model(vocab_size, e_dim, rnn_units, 1)
model_pred.load_weights(tf.train.latest_checkpoint(ckpt))

In [ ]:
pred_text(model_pred, "ROMEO:", 100)

'ROMEO:Hais grant; you have left upon God for it:And, after, my groancidUEEE:Ay, and If thou do mischance!W'

In [ ]:
def pred_text(model, s, num=1000):
  input_eval = [char2idx[i] for i in s]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0

  model.reset_states()
  for i in range(num):
    pred = model(input_eval)
    pred = tf.squeeze(pred, 0) / temperature
    pred_id = tf.random.categorical(pred, 1)[-1,0].numpy()

    input_eval = tf.expand_dims([pred_id], 0)
    # input_eval = tf.concat([input_eval, input_eval1], 1)
    text_generated.append(idx2char[pred_id])

  return s + ''.join(text_generated)


In [ ]:
pred_text(model_pred, 'Romeo', 100)

"RomeouNyNys.ggegususesA A'?Rgh iqgmiyUS:S:S:an O OBOBOBOBOBOfLf ffbwlwlgpd uWfwL Ldsdh's:t:Z.Fror;To:D:D:"

In [ ]:
a = [1,2]
b = tf.expand_dims(a, 0)
b1 = tf.expand_dims([3], 0)
tf.concat([b,b1],1)

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]], dtype=int32)>